# Summary

This notebook combines and validates daily price datasets downloaded using the bulk API offered by Financial Modeling Prep (FMP): https://site.financialmodelingprep.com/developer/docs#bulk-and-batch

# Imports and Configuration

In [1]:
import os
import glob
import pandas as pd

from tqdm import tqdm

In [2]:
# this folder contains .csv datasets of daily prices
INPUT_PATH = r"C:\Users\mushj\Downloads\RAW FINANCE DATA\FMP\bulk_batch-historical-eod"
OUTPUT_PATH = r"C:\Users\mushj\Downloads\PROCESSED FINANCE DATA\FMP"

# Examine data

In [3]:
# get paths to all daily prices datasets
dataset_names = glob.glob(os.path.join(INPUT_PATH, '*'), recursive=False)
print("Counts (number of days):", len(dataset_names))
print("Example path:", dataset_names[0])

Counts (number of days): 5217
Example path: C:\Users\mushj\Downloads\RAW FINANCE DATA\FMP\bulk_batch-historical-eod\batch-historical-eod (2005-01-03).csv


In [4]:
# load one example to inspect the schema
sample_df = pd.read_csv(dataset_names[0])
sample_df.head()

,symbol,date,open,low,high,close,adjClose,volume
0,1198.HK,2005-01-03,4.590,4.52,4.590,4.56,4.56,190692.0
1,DHCC,2005-01-03,0.570,0.55,0.630,0.63,0.63,10200.0
2,CALZF,2005-01-03,0.950,0.95,0.950,0.95,0.95,500.0
3,BZ7A.F,2005-01-03,0.290,0.29,0.290,0.29,0.29,0.0
4,TCOR,2005-01-03,0.035,0.03,0.035,0.03,0.03,114040.0


In [5]:
sample_df.shape

(18866, 8)

In [6]:
sample_df.dtypes

symbol       object
date         object
open        float64
low         float64
high        float64
close       float64
adjClose    float64
volume      float64
dtype: object

# Load and validate all datasets

In [7]:
# check each dataset for the following columns
expected_columns = {'symbol', 'date', 'open', 'low', 'high', 'close', 'adjClose', 'volume'}

# only keep the following columns
keep_columns = ['symbol', 'date', 'close', 'volume']

# list to store each dataset
df_list = []

for file in tqdm(dataset_names, desc="Loading datasets"):
    df = pd.read_csv(file)
    
    # raise exception if set of columns don't match expected set
    if set(df.columns) != expected_columns:
        file_name = dataset_names[0].split('\\')[-1]
        error_msg = f"Unexpected columns found in {file_name}"
        error_msg += f"\nColumns: {df.columns}"
        raise Exception(error_msg)
    
    df_list.append(df[keep_columns])

Loading datasets: 100%|████████████████████████████████████████████████████████████| 5217/5217 [03:46<00:00, 23.08it/s]


In [11]:
%%time
# union all and sort
combined_df = (
    pd.concat(df_list, ignore_index=True, axis=0)
    .sort_values(['symbol', 'date'])
)
combined_df.shape

(240423841, 4)

In [15]:
%%time
# export combined and sorted dataset
combined_df.to_csv(OUTPUT_PATH+'/FMP_daily_prices.csv', index=False)

CPU times: total: 4min 12s
Wall time: 7min 2s


# Get subset

In [ ]:
# get SP-500 list of symbols
sp_500 = pd.read_html("https://en.wikipedia.org/wiki/List_of_S%26P_500_companies")[0]
sp_500_symbols = set(sp_500.Symbol)

# get Russell 1000 list of symbols
russell_1000 = pd.read_html("https://en.wikipedia.org/wiki/Russell_1000_Index")[3]
russell_1000_symbols = set(russell_1000.Symbol)

# get NASDAQ 100 list of symbols
nasdaq_100 = pd.read_html("https://en.wikipedia.org/wiki/Nasdaq-100")[4]
nasdaq_100_symbols = set(nasdaq_100.Symbol)

# union all
all_symbols = sp_500_symbols.union(russell_1000_symbols).union(nasdaq_100_symbols)

print('SP500:', len(sp_500_symbols))
print('russell_1000:', len(russell_1000_symbols))
print('nasdaq_100:', len(nasdaq_100_symbols))
print('union:', len(all_symbols))

In [ ]:
# filter and export
%%time
(
    combined_df
    .query('symbol in @all_symbols')
    .to_csv(OUTPUT_PATH+'/FMP_daily_prices_top1k.csv', index=False)
)